In [6]:
from PIL import Image
import os
import pathlib
import cv2
import numpy as np
import torch
from torch.utils.data import Dataset
import random
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle as sk_shuffle
import time
import os

import matplotlib.pyplot as plt

from torch.utils.data.sampler import SubsetRandomSampler, SequentialSampler

import glob2
import pdb
import ipdb

In [7]:

SEED = 42


labels_dict = {'1':1, '2':2, '3':3, '4':4, '5':5,
               '6':6, '7':7, '8':8, '9':9, '10':10,
               '11':11, '12':12, '13':13, '14':14, '15':15,
               '16':16, '17':17, '18':18, '19':19, '20':20,
               '21':21, '22':22, '23':23, '24':24, '25':25,
               '26':26, '27':27, '28':28, '29':29, '30':30,
               '31':31, '32':32, 'a':33, 'b':34, 'c':35, 'd':36,
               'e':37, 'f':38, 'g':39, 'h':40, 'i':41, 'j':42, 'k':43,
               'l':44, 'm':45, 'n':46, 'o':47, 'p':48, 'q':49, 'r':50, 's':51, 't':52,
               'conecut':53, 'caries':54, 'caries2':54,'caries3':54,'crown':55, 'bridge':56,
               'gutta-percha':57, 'pin':58, 'post':59, 'buildup':60, 'calculus':61, 'sealer':62,
               'cement':63, 'bracket':64, 'retainer':65, 'instrument':66, 'implant':67, 'screw':68,
               'veneer':69, 'silver-point':70, 'space-maintainer':71, 'core':72, 'base':73,
               'temporary-filling':74, 'medicament':75, 'framework':76, 'liner':77, 'onlay-composite':78,
               'onlay-metal':79, 'onlay-ceramic':80, 'inlay-ceramic':81, 'inlay-composite':82, 'inlay-metal':83,
               'filling-composite':84, 'filling-glass':85, 'filling-metal':86}

reverse_labels_dict = {v: k for k, v in labels_dict.items()}

labels_dict_no_baby = {'1':1, '2':2, '3':3, '4':4, '5':5,
               '6':6, '7':7, '8':8, '9':9, '10':10,
               '11':11, '12':12, '13':13, '14':14, '15':15,
               '16':16, '17':17, '18':18, '19':19, '20':20,
               '21':21, '22':22, '23':23, '24':24, '25':25,
               '26':26, '27':27, '28':28, '29':29, '30':30,
               '31':31, '32':32, 'conecut':33, 'caries':34, 'caries2':34,'caries3':34}
regions_dict_all = {'Anterior1MandibleB':0,
               'Anterior2MaxillaL':1,
               'Anterior2MaxillaR':2,
               'Anterior3MandibleL':3,
               'Anterior3MandibleR':4,
               'Anterior3MaxillaL':5,
               'Anterior3MaxillaR':6,
               'Molar3JawregionL':7,
               'Molar3JawregionR':8,
               'Molar3MandibleL':9,
               'Molar3MandibleR':10,
               'Molar3MaxillaL':11,
               'Molar3MaxillaR':12,
               'Anterior1MandibleL':13,
               'Premolar2JawregionR':14,
               'Premolar2MandibleR':15,
               'Premolar2MaxillaL':16,
               'Premolar2MaxillaR':17,
               'Anterior1MaxillaB':18,
               'Premolar2MandibleL':19,
               'Premolar2JawregionL':20,
               'Anterior1MaxillaR':21,
               'Anterior1MaxillaL':22,
               'Anterior2MaxillaB':23,
               'Anterior2MandibleL':24,
               'Anterior1MandibleR':25}

regions_dict_standard = {'Anterior2MaxillaR':0,
                         'Anterior3MaxillaR':1,
                         'Anterior1MandibleB':2,
                         'Molar3MandibleR':3,
                         'Molar3MaxillaL':4,
                         'Anterior2MaxillaL':5,
                         'Anterior3MaxillaL':6,
                         'Molar3MaxillaR':7,
                         'Anterior3MandibleL':8,
                         'Premolar2MaxillaL':9,
                         'Premolar2MandibleR':10,
                         'Premolar2MandibleL':11,
                         'Molar3JawregionL':12,
                         'Anterior3MandibleR':13,
                         'Molar3JawregionR':14,
                         'Premolar2JawregionL':15,
                         'Premolar2JawregionR':16,
                         'Premolar2MaxillaR':17,
                         'Anterior1MaxillaB':18,
                         'Molar3MandibleL':19}


In [25]:
tooth_dict = {'1':1, '2':2, '3':3, '4':4, '5':5,
               '6':6, '7':7, '8':8, '9':9, '10':10,
               '11':11, '12':12, '13':13, '14':14, '15':15,
               '16':16, '17':17, '18':18, '19':19, '20':20,
               '21':21, '22':22, '23':23, '24':24, '25':25,
               '26':26, '27':27, '28':28, '29':29, '30':30,
               '31':31, '32':32, 'a':33, 'b':34, 'c':35, 'd':36,
               'e':37, 'f':38, 'g':39, 'h':40, 'i':41, 'j':42, 'k':43,
               'l':44, 'm':45, 'n':46, 'o':47, 'p':48, 'q':49, 'r':50, 's':51, 't':52}
prior_dict = {'crown':1, 'bridge':2,
               'gutta-percha':3, 'pin':4, 'post':5, 'buildup':6, 'calculus':7, 'sealer':8,
               'cement':9, 'bracket':20, 'retainer':11, 'instrument':12, 'implant':13, 'screw':14,
               'veneer':15, 'silver-point':16, 'space-maintainer':17, 'core':18, 'base':19,
               'temporary-filling':20, 'medicament':21, 'framework':22, 'liner':23, 'onlay-composite':24,
               'onlay-metal':25, 'onlay-ceramic':26, 'inlay-ceramic':27, 'inlay-composite':28, 'inlay-metal':29,
               'filling-composite':30, 'filling-glass':31, 'filling-metal':32}

filling_dict = {'temporary-filling':1, 'filling-composite':2, 'filling-glass':3, 'filling-metal':4}
reverse_filling_dict = {v: k for k, v in filling_dict.items()}


tooth_list = list(tooth_dict.keys())

# oversample_filter = lambda x: 'calculus' in x and x.endswith('.jpg')

class CalculusBase:
    def __init__(self, path, segregate_label, oversample_label=None, oversample_factor=None, label_filter=lambda x: x.endswith('.jpg'),
                 im_range = [0, 55000] ,issynthetic=False, test_size=0.2, isTrain=True):
        if not path.endswith('train'): path = os.path.join(path, 'train')
        path = os.path.join(path, 'images')
        path = pathlib.Path(path)
        self.isTrain = isTrain
        img_paths=[]
        
        if not issynthetic:
            # use only standard views
            for std_view in regions_dict_standard:
                pathk = '*/' + std_view + '/im.jpg'
                img_paths += list(path.glob(pathk))
        else:
            # use only standard views
            for std_view in regions_dict_standard:
                pathk = '*/' + std_view + '/*/im.jpg'
                img_paths += list(path.glob(pathk))
        
        in_impaths = img_paths if im_range=='all' else img_paths[im_range[0]:im_range[1]]

        assert callable(label_filter)
        self.label_filter=label_filter
        self.sg_label = segregate_label
        self.sg_filter = lambda x: self.sg_label in x and x.endswith('.jpg')
        self.oversample_label = oversample_label
        self.oversample_factor = oversample_factor
        
        lbpaths = []
        impaths = []
        self.all_impaths = in_impaths #self.train_impaths if self.isTrain else self.val_impaths
        
        print('All Input Images = {}'.format(len(self.all_impaths)))
        bg_imgs, bg_lbls, fg_imgs, fg_lbls = self.segregate()
        print('All BG Images = {}'.format(len(bg_imgs)))
        print('All FG Images = {}'.format(len(fg_imgs)))
        self.train_fgimgs, self.val_fgimgs, self.train_fglbls, self.val_fglbls = train_test_split(fg_imgs, fg_lbls, test_size= test_size, random_state=42)
        over_imgs, over_lbls = self.oversample()
        
        if len(bg_imgs) > len(over_imgs)+len(self.val_fgimgs):
            bg_imouts = bg_imgs[:len(over_imgs)+len(self.val_fgimgs)]
            bg_lblouts = bg_lbls[:len(over_imgs)+len(self.val_fgimgs)]
        else:
            bg_imouts = bg_imgs
            bg_lblouts = bg_lbls
        
        train_bgimgs, val_bgimgs, train_bglbls, val_bglbls = train_test_split(bg_imouts, bg_lblouts, test_size= test_size, random_state=42)
        
        train_impaths = train_bgimgs + over_imgs
        train_lbpaths = train_bglbls + over_lbls
        self.train_impaths, self.train_lbpaths = sk_shuffle(train_impaths, train_lbpaths, random_state=42)
        
        val_impaths = val_bgimgs + self.val_fgimgs
        val_lbpaths = val_bglbls + self.val_fglbls
        self.val_impaths, self.val_lbpaths = sk_shuffle(val_impaths, val_lbpaths, random_state=42)
        
    def segregate(self):
        
        bg_img = []
        bg_lbl = []
        fg_img = []
        fg_lbl = []
        
        for impath in self.all_impaths:
            impath = impath.as_posix()
            lbpath = os.path.dirname(impath).replace('images', 'masks', 1)
            lbpath_dict = {self.sg_label:[]}
            sg_lbpath = [os.path.join(lbpath, lbname) for lbname in os.listdir(lbpath) if self.sg_filter(lbname)]
            if len(sg_lbpath) > 0:
                lbpath_dict[self.sg_label].extend(sg_lbpath)
                fg_img.append(impath)
                fg_lbl.append(lbpath_dict)
            else:
                bg_img.append(impath)
                bg_lbl.append(lbpath_dict)
        
        return bg_img, bg_lbl, fg_img, fg_lbl
        
    def oversample(self):
        
        over_imgs, over_lbls = [], []
        if isinstance(self.oversample_factor, int) and self.oversample_factor >= 0 and self.oversample_label is not None:
            for impath, lbpath_dict in zip(self.train_fgimgs, self.train_fglbls):
                over_imgs.append(impath)
                over_lbls.append(lbpath_dict)
                class_names = [m.split('/')[-1].split('.')[0] for m in lbpath_dict[self.sg_label]]
                if self.oversample_label in class_names:
                    over_lbls += [lbpath_dict]*self.oversample_factor
                    over_imgs += [impath]*self.oversample_factor
            over_imgs, over_lbls = sk_shuffle(over_imgs, over_lbls, random_state=42)
            return over_imgs, over_lbls
        else:
            return self.train_fgimgs, self.train_fglbls
        
    
    def get_indices(self):
        return self.train_idxs if self.isTrain else self.val_idxs
    
    def __len__(self):
        return len(self.train_impaths) + len(self.val_impaths)
    
    def __getitem__(self, idx):
        if self.isTrain:
            return self.train_impaths[idx], self.train_lbpaths[idx]
        else:
            return self.val_impaths[idx], self.val_lbpaths[idx]
    
mark1=time.time()
filling_test = CalculusBase('/home/rohan/Datasets/prior_clean/train', segregate_label='calculus', oversample_label='calculus', oversample_factor=0, 
                            issynthetic=False, im_range = 'all', test_size=0.001, isTrain=True)
print('dataset time = {} secs'.format(time.time()-mark1)) 


All Input Images = 60178
All BG Images = 46422
All FG Images = 13756
dataset time = 3.9577314853668213 secs


In [32]:
filling_test[9]

('/home/rohan/Datasets/prior_clean/train/images/1.1909476924026491962.003750907121712/Anterior1MandibleB/im.jpg',
 {'calculus': ['/home/rohan/Datasets/prior_clean/train/masks/1.1909476924026491962.003750907121712/Anterior1MandibleB/calculus.jpg']})